In [ ]:
import inspect
import sys
import bookutils
import ipywidgets as widgets

In [ ]:
from Tracer import Tracer
from bookutils import input,next_inputs

In [ ]:
class State():
    def __init__(self,event,abs_lineno,code,changed_vars,call_stack,frame,rel_lineno,fun_name):
        self.event = event
        self.abs_lineno = abs_lineno
        self.rel_lineno = rel_lineno
        self.code = code
        self.changed_vars = changed_vars
        self.call_stack = call_stack
        self.frame = frame
        self.stackdepth = None
        self.fun_name = fun_name

    def __str__(self):
        return 'Line: ' + str(self.abs_lineno) + ' ('+ str(self.rel_lineno)+'), ('+repr(self.event)+") " + repr(self.code) + '   '+ repr(self.changed_vars) + '   ' + repr(self.fun_name)

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
class Tracer(object):
    def __init__(self, file=sys.stdout):
        """Trace a block of code, sending logs to file (default: stdout)"""
        self.original_trace_function = None
        self.file = file
        self.loglist =[]
        self.last_vars = {}
        pass

    def changed_vars(self, new_vars):
        changed = {}
        for var_name in new_vars:
            if (var_name not in self.last_vars or
                    self.last_vars[var_name] != new_vars[var_name]):
                changed[var_name] = new_vars[var_name]
        self.last_vars = new_vars.copy()
        return changed

    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        # print(*objects, sep=sep, end=end, file=self.file, flush=True)
        # self.logger = self.logger + "\n" + " ".join(objects)


        current_state = State(objects[0], objects[1],objects[2],objects[3],None,objects[5],objects[6],objects[7])
        self.loglist.append(current_state)


    def traceit(self, frame, event, arg):
        """Tracing function. To be overridden in subclasses."""
        self.log_state(frame, event, arg)
        #self.log(event, frame.f_lineno, frame.f_code.co_name, frame.f_locals)

    def log_state(self, frame, event, arg):

        changes = self.changed_vars(frame.f_locals)
        module = inspect.getmodule(frame.f_code)

        if module is None:
            source,function_start = inspect.getsourcelines(frame.f_code)
        else:
            source,function_start = inspect.getsourcelines(module)

        relative_lineno = frame.f_lineno-function_start + 1

        if event == 'call':
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes, None, frame, relative_lineno,frame.f_code.co_name)

        if event == 'line':
            current_line = source[relative_lineno-1]
            self.log(event, frame.f_lineno, current_line, changes, None, frame, relative_lineno,frame.f_code.co_name)

        if event == 'return':
            #TODO do we even want this as a  state
            self.log(event, frame.f_lineno, frame.f_code.co_name, changes,None, frame, relative_lineno,frame.f_code.co_name)
            self.last_vars = {}  # Delete 'last' variables


    def _traceit(self, frame, event, arg):
        """Internal tracing function."""
        if frame.f_code.co_name == '__exit__':
            # Do not trace our own _exit_() method
            pass
        else:
            self.traceit(frame, event, arg)
        return self._traceit

    def __enter__(self):
        """Called at begin of `with` block. Turn tracing on."""
        self.original_trace_function = sys.gettrace()
        sys.settrace(self._traceit)
        return self

    def __exit__(self, tp, value, traceback):
        """Called at begin of `with` block. Turn tracing off."""
        sys.settrace(self.original_trace_function)
        #self.print_log()

    def print_log(self):
        index = 0
        for log in self.loglist:
            #print('Log Entry: ' + str(index))
            #index = index + 1
            print(log)

    def get_log(self):
        return self.loglist

In [ ]:
class Debugger(object):
    def __init__(self):

        self.current_state = None
        self.callstack = None
        self.current_step = 0
        self.stepping = True
        self.breakpoints = set()
        self.interact = True

        self.variables = {}

        #self.frame = None
        #self.event = None
        #self.arg = None

    def __enter__(self):
        with Tracer() as t:
            foo()
        self.execution_log = t.get_log()
        t.print_log()
        print("\n")
        print("\n")
        print("\n")
        if len(self.execution_log)<1:
            print("Log is empty")
            return
        self.construct_callstack()
        self.current_state=self.execution_log[self.current_step]
        #self.execution_loop()
        self.print_debugger_status()
        self.interaction_loop()

    def __exit__(self, exc_type, exc_val, exc_tb):
        pass

    def execution_loop(self):
        for state in self.execution_log:
            self.current_state = state
            self.construct_vars()
            if self.stop_here():
                self.print_debugger_status()
                self.interaction_loop()

        print("Program finished")


    def step(self):
        if self.current_step<len(self.execution_log)-1:
            self.current_step += 1
            self.current_state = self.execution_log[self.current_step]
            self.construct_vars()
            return True
        else:
            return False

    def step_back(self):
        if self.current_step> 0:
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
            return True
        else:
            return False

    def stop_here(self):
        """Return true if we should stop"""
        return self.stepping or self.current_state.abs_lineno in self.breakpoints

    def construct_callstack(self):

        callstack=list()
        currentdepth = -1

        for i in range(0,len(self.execution_log)):
            currentexecution = self.execution_log[i]

            if currentexecution.event == 'call':
                #self.log(currentexecution)
                callstack.append(currentexecution)
                currentdepth = currentdepth + 1

            elif currentexecution.event == 'return':
                currentdepth = currentdepth - 1

            currentexecution.stackdepth = currentdepth

        self.callstack=callstack

    def backconstruct_vars(self):
        backconstructed_vars = dict()

        if(self.current_step>0):
            num_subroutines = 0
            for i in range(0,self.current_step):
                current_execution = self.execution_log[self.current_step-(i+1)]
                if current_execution.event == 'return':
                    num_subroutines  +=1
                elif current_execution.event == 'call':
                    if num_subroutines == 0: break
                    num_subroutines -=1
                elif num_subroutines==0:
                    changed_vars = current_execution.changed_vars.copy()
                    changed_vars.update(backconstructed_vars)
                    backconstructed_vars = changed_vars

            self.variables = backconstructed_vars
            return  True
        else:
            return False



    def construct_vars(self):
        event = self.current_state.event
        if event == 'return':
            self.variables.clear() #TODO this deletes too early
        elif event == 'call':
            self.variables = self.current_state.changed_vars
        else:
            self.variables.update(self.current_state.changed_vars)

    def interaction_loop(self):
        self.interact = True
        while self.interact:
            command = input("(debugger) ")
            self.execute(command)

    def commands(self):
        cmds = [method.replace('_command', '')
                for method in dir(self.__class__)
                if method.endswith('_command')]
        cmds.sort()
        return cmds

    def command_method(self, command):
        if command.startswith('#'):
            return None  # Comment

        possible_cmds = [possible_cmd for possible_cmd in self.commands()
                         if possible_cmd.startswith(command)]
        if len(possible_cmds) != 1:
            self.help_command(command)
            return None

        cmd = possible_cmds[0]
        return getattr(self, cmd + '_command')

    def execute(self, command):
        sep = command.find(' ')
        if sep > 0:
            cmd = command[:sep].strip()
            arg = command[sep + 1:].strip()
        else:
            cmd = command.strip()
            arg = ""

        method = self.command_method(cmd)
        if method:
            method(arg)

    def next_command(self,arg=""):
        """Step over function calls going to the next line"""


        fun_name = self.current_state.fun_name

        if (self.current_state.event == 'call'):
            while not (self.current_state.event == 'return' and self.current_state.fun_name == fun_name):
                self.step()
        self.step()
        self.print_debugger_status()

    def previous_command(self,arg=""):
        """self.print_debugger_status"""
        fun_name = self.current_state.fun_name
        self.step_back()

        if self.current_state.event!='call':
            while not (self.current_state.fun_name == fun_name and self.current_state.event=='line'):
                self.step_back()
        else:
            self.step_back()

        self.backconstruct_vars()
        self.print_debugger_status()

    def next_state(self):
        if self.current_step<len(self.execution_log)-1:
            return self.execution_log[self.current_step+1]
        else:
            return None

    def until_command(self,arg=""):
        if not arg:
            #no arg case
            self.until_line(self.current_state.abs_lineno,None)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            try:
                target_line = int(secondary_arg)
            except ValueError:
                function_name=secondary_arg
            if target_line > 0:
                self.until_line(target_line,filename)
            else:
                self.until_function(function_name,filename)
        else:
            #simple arg case
            try:
                target_line = int(arg)
            except ValueError:
                function_name=arg

            if function_name:
                self.until_function(function_name,None)
            else:
                self.until_line(target_line,None)


    def backuntil_command(self,arg=""):
        if not arg:
            #no arg case
            self.backuntil_line(self.current_state.abs_lineno,None)
            return

        function_name=""
        target_line=0
        sep = arg.find(':')
        if sep > 0:
            #compound arg case
            filename = arg[:sep].strip()
            secondary_arg = arg[sep+1:].strip()
            try:
                target_line = int(secondary_arg)
            except ValueError:
                function_name=secondary_arg
            if target_line > 0:
                self.backuntil_line(target_line,filename)
            else:
                self.backuntil_function(function_name,filename)
        else:
            #simple arg case
            try:
                target_line = int(arg)
            except ValueError:
                function_name=arg

            if function_name:
                self.backuntil_function(function_name,None)
            else:
                self.backuntil_line(target_line,None)

    def until_function(self,target_function,target_file):
        """Advance until call to target_function and in target_file if given"""
        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()

        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" ):
            stepped = self.step()
            if not stepped:
                break

        if stepped:
            #self.backstep_command()
            self.print_debugger_status()
        else:
            self.log("Couldn't find any execution of function named:"+ str(target_function))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()

    def backuntil_function(self,target_function,target_file):
        """Run backwards until call to target_function and in target_file if given"""
        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()

        stepped = True
        while not (self.current_state.fun_name == target_function and self.current_state.event=="call" ):
            stepped = self.step_back()
            if not stepped:
                break

        if stepped:
            self.backconstruct_vars()
            self.print_debugger_status()
        else:
            self.log("Couldn't find any execution before current line of function named:"+ str(target_function))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()
        
    def until_line(self,target_line,target_file):
        """Advance until program line is greater than target_line and in target_file if given"""
        #TODO add target_file functionality
        if target_line<=0:
            self.log("Invalid line: " + str(target_line))
            return

        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()
        stepped=False
        while self.current_state.abs_lineno <= target_line:
            stepped=self.step()
            if not stepped:
                break
                
        if stepped:
            self.print_debugger_status()
        else:
            self.log("Couldn't reach any line greater than "+ str(target_line))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()

    def backuntil_line(self,target_line,target_file):
        """Run backwards until program line is lower than target_line and in target_file if given"""
        #TODO add target_file functionality
        if target_line<=1:
            self.log("Invalid line: " + str(target_line))
            return

        start_state = self.current_state
        start_step = self.current_step
        start_variables = self.variables.copy()
        stepped=False
        while self.current_state.abs_lineno >= target_line:
            stepped=self.step_back()
            if not stepped:
                break
    
        if stepped:
            self.backconstruct_vars()
            self.print_debugger_status()
        else:
            self.log("Couldn't reach any line lower than "+ str(target_line))
            self.current_state = start_state
            self.current_step = start_step
            self.variables=start_variables
            self.print_debugger_status()
            

    def finish_command(self,arg=""):
        """Execute until return of current function"""
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="return"):
            self.step()
        self.print_debugger_status()

    def start_command(self,arg=""):
        """Execute backwards until a function start"""
        fun_name = self.current_state.fun_name
        while not (self.current_state.fun_name == fun_name and self.current_state.event=="call"):
            self.current_step -= 1
            self.current_state = self.execution_log[self.current_step]
        self.variables = self.current_state.changed_vars.copy() #TODO what about member variables
        self.print_debugger_status()

    def where_command(self,arg=""):
        currentstate = self.execution_log[self.current_step]
        currentdepth = currentstate.stackdepth
        for i in range (0,len(self.callstack)):
            if i!=currentdepth:
                self.log(str(self.callstack[i]))
            else:
                self.log('     > ' + str(self.callstack[i]))

    def has_breakpoint(self):
        return self.current_state.abs_lineno in self.breakpoints

    def quit_command(self,arg=""):
        """End the debugger session"""
        self.interact=False
        self.log("Quit debugger session")

    def continue_command(self, arg=""):
        """Resume execution"""
        stepping = True
        while stepping:
            stepped = self.step()
            if not stepped:
                self.interact=False
                stepping = False
            if self.has_breakpoint():
                stepping = False

        self.print_debugger_status()

    def reverse_command(self, arg=""):
        while not (self.current_state.abs_lineno in self.breakpoints or self.current_step==0):
                self.step_back()
        self.print_debugger_status()

    def backstep_command(self,arg=""):
        """Step to the previous executed line"""
        status = self.backconstruct_vars()
        if status:
            self.current_step-=1
            self.current_state = self.execution_log[self.current_step]
        self.print_debugger_status()

    def step_command(self, arg=""):
        """Execute up to the next line"""
        self.step()
        self.print_debugger_status()

    def print_command(self, arg=""):
        """Print an expression. If no expression is given, print all variables"""
        #vars = self.current_state.frame.f_locals
        vars = self.variables
        if not arg:
            self.log("\n".join([f"{var} = {repr(vars[var])}" for var in vars]))
        else:
            try:
                self.log(f"{arg} = {repr(eval(arg, None, vars))}")
            except Exception as err:
                self.log(f"{err.__class__.__name__}: {err}")

    def break_command(self, arg=""):
        """Set a breakpoint in given line. If no line is given, list all breakpoints"""
        if arg:
            self.breakpoints.add(int(arg))
        self.log("Breakpoints:", self.breakpoints)

    def delete_command(self, arg=""):
        """Delete breakpoint in given line. Without given line, clear all breakpoints"""
        if arg:
            try:
                self.breakpoints.remove(int(arg))
            except KeyError:
                self.log(f"No such breakpoint: {arg}")
        else:
            self.breakpoints = set()
        self.log("Breakpoints:", self.breakpoints)

    def print_debugger_status(self):
        frame = self.current_state.frame
        event = self.current_state.event
        changes = self.current_state.changed_vars
        changes_s = ", ".join([var + " = " + repr(changes[var])
                               for var in changes])

        if event == 'call':
            self.log("Calling " + frame.f_code.co_name + '(' + changes_s + ')')
        #elif changes:
            #self.log(' ' * 40, '#', changes_s)

        # if event == 'line':
        #     module = inspect.getmodule(frame.f_code)
        #     if module is None:
        #         source = inspect.getsource(frame.f_code)
        #     else:
        #         source = inspect.getsource(module)
        #     current_line = source.split('\n')[self.current_state.lineno - 1]
        #     self.log(repr(self.current_state.lineno) + ' ' + current_line)

        if event == 'line':
            current_line = self.current_state.code
            self.log(repr(self.current_state.abs_lineno) + ' ' + current_line)
        if event == 'return':
            self.log(frame.f_code.co_name + '()' + " returns " ) #used to have +repr(arg)
            #self.last_vars = {}  # Delete 'last' variables



    def log(self, *objects, sep=' ', end='\n', flush=False):
        """Like print(), but always sending to file given at initialization,
           and always flushing"""
        print(*objects, sep=sep, end=end, file=sys.stdout, flush=True)

    def help_command(self, command=""):
        """Give help on given command. If no command is given, give help on all"""
        if command:
            possible_cmds = [possible_cmd for possible_cmd in self.commands()
                             if possible_cmd.startswith(command)]

            if len(possible_cmds) == 0:
                print(f"Unknown command {repr(command)}. Possible commands are:")
                possible_cmds = self.commands()
            elif len(possible_cmds) > 1:
                print(f"Ambiguous command {repr(command)}. Possible expansions are:")
        else:
            possible_cmds = self.commands()

        for cmd in possible_cmds:
            method = self.command_method(cmd)
            print(f"{cmd:10} -- {method.__doc__}")

In [ ]:
def foo():
    a = 12
    bar(21)
    x = 5
    a = 11

    return

def bar(arg):
    c = arg
    y = 13
    a = 1
    code()

def code():
    return



In [ ]:
with Debugger():
     pass



In [ ]:
#d= Debugger(res)
#d.start()
#next_inputs(["step","print","step","step","print","step","print s","continue"])
#next_inputs(["step","step","step","step","step","step","continue"])
#next_inputs(["step","break 4","continue","continue"])
#with Debugger():
#    remove_html_markup("<b>bold</b>")

In [ ]:
def create_expanded_button(description, button_style):
    return widgets.Button(description=description, button_style=button_style, layout=widgets.Layout(height='auto', width='auto'))


play = widgets.Play(
#     interval=10,
    value=0,
    min=0,
    max=100,
    step=1,
    description="Press play",
    disabled=False
)
slider = widgets.IntSlider()
widgets.jslink((play, 'value'), (slider, 'value'))
tester = widgets.HBox([play, slider])


top_left_button = create_expanded_button("Top left", 'info')
top_right_button = create_expanded_button("Top right", 'success')
bottom_left_button = create_expanded_button("Bottom left", 'danger')
bottom_right_button = create_expanded_button("Bottom right", 'warning')

top_left_text = widgets.IntText(description='Top left', layout=widgets.Layout(width='auto', height='auto'))
top_right_text = widgets.IntText(description='Top right', layout=widgets.Layout(width='auto', height='auto'))
bottom_left_slider = widgets.IntSlider(description='Bottom left', layout=widgets.Layout(width='auto', height='auto'))
bottom_right_slider = widgets.IntSlider(description='Bottom right', layout=widgets.Layout(width='auto', height='auto'))

header_button = create_expanded_button('Header', 'success')
left_button = create_expanded_button('Left', 'info')
center_button = create_expanded_button('Center', 'warning')
right_button = create_expanded_button('Right', 'info')
footer_button = create_expanded_button('Footer', 'success')

widgets.AppLayout(header=tester,
          center=center_button,
          right_sidebar=right_button,
          pane_widths=[2, 0.5, 1],
          pane_heights=[1, 17, 1])


